# Rendu Projet Maths/Info III 
#### Clément Desroches & Mathis Bourdin

Etant donné la présence du notebook à remplir, il a été choisi de suivre celui-ci pour le rendu de ce projet.
La démarche ne sera alors pas globale comme il a été fait pour les précédents projets. Nous répondrons simplement question par question.

# Enoncé du problème

L'objectif de ce projet est d'estimer la longueur de câble sous-marin nécessaire pour relier deux côtes $A$ et $B$  en utilisant des simulations conditionnelles.


Le câble reposera sur le fond marin dont la profondeur est inconnue.
Le segment $[AB]$ est discrétisé par une séquence de (N+1) points. On pose $x_0=A$ et pour $i=1,\dots,N$, $$x_i=x_0+i\Delta$$ où $$\Delta = \frac{AB}{N}$$ de telle sorte que $x_N=B$.
On note $z(x)$ la profondeur du fond marin au point $x$ de telle sorte 
qu'on pourra estimer la longueur totale de câble nécessaire par la somme 
des longueurs sur les segments de la discrétisation :

$$l=\sum_{i=1}^N\sqrt{\Delta^2+(z(x_i)-z(x_{i-1}))^2}.$$

Enfin, notons que l'on dispose d'un ensemble de $n$ observations de la 
profondeur que l'on supposera situées sur des points de discrétisation $z(x_{j_1}),\dots,z(x_{j_n})$.


On adopte un modèle probabiliste pour la profondeur. On suppose que le vecteur des 
profondeurs sur les points de discrétisation 
$\mathbf{z}=(z(x_0),\dots,z(x_N))$ est la réalisation
d'un vecteur aléatoire gaussien $\mathbf{Z}=(Z(x_0),\dots,Z(x_N))$ 
dont le vecteur d'espérance ne contient qu'une seule valeur $\mu$ 
répétée $N+1$ fois et dont la matrice de covariance $\Sigma$ a pour termes $\sigma_{ij}$
définis par $\sigma_{ij}=C(|x_i-x_j|)$ où $C$ est une
fonction décroissante, traduisant le fait que deux points 
géographiquement proches ont tendance à avoir des profondeurs plus similaires que deux points éloignés.

On supposera que la matrice de covariance ainsi 
générée est définie-positive (en fait, $C$ sera choisie parmi les fonctions qui, 
appliquées aux termes d'une matrice de distance, produisent des matrices définie-positives). 

Si on note $L$ la variable aléatoire donnant la longueur de cable nécessaire : 
$$L=\sum_{i=1}^N\sqrt{\Delta^2+(Z(x_i)-Z(x_{i-1}))^2},$$
un bon estimateur de $L$ est fourni par l'espérance conditionnelle 

$$L^\star=E[L|Z(x_{j_1})=z(x_{j_1}),\dots,Z(x_{j_n})=z(x_{j_n})].$$
                                                                              
Cependant, cette quantité est difficilement accessible par le calcul. 
On va donc avoir recours à des
simulations conditionnelles. C'est-à-dire que l'on va simuler 
un nombre $K$ de réalités (disons des réalisations du modèle 
probabiliste choisi), et sur chacune d'entre elle, 
la quantité de câble nécessaire sera évaluée. 
On disposera ainsi d'un échantillon $l_{(1)},\dots,l_{(K)}$ de 
longueures simulées. Puis on approchera l'espérance conditionnelle  par 
$$L^\star=\sum_{k=1}^K l_{(k)}.$$

L'objectif de ce projet est donc d'écrire un code permettant 
d'effectuer cette simulation conditionnelle, puis de l'appliquer 
au jeu de données fourni et d'en déduire une estimation de la longueur de câble nécessaire.

# Questions théoriques

1. Quel théorème du cours nous autorise-t-il à estimer l'espérance conditionnelle par la moyenne empirique de simulations conditionnelles ?

2. Rappeler la loi conditionnelle du vecteur des composantes de $\mathbf{Z}$ correspondant aux points de discrétisation
sans observation, connaissant les valeurs prises par les composantes aux sites d'observation.

3. Si $\mathbf{Y}=(Y_1,\dots,Y_p)$ est un vecteur de composantes gaussiennes indépendantes, toutes d'espérance nulle et de variance 1, 
quelle est la loi du vecteur $\mathbf{Z}=m+R\mathbf{Y}$ où $R$ est une matrice $p\times p$ et $m$ est un vecteur de taille $p$ ?

4. En déduire un algorithme de simulation conditionnelle.

##### 1.
Le théorème qui permet d'estimer l’espérance conditionnelle par la moyenne empirique de simulations conditionnelles est la loi faible des grands nombres. D'après cette loi, en choisissant un K assez grand, La moyenne empirique des simulations va converger vers l'espérance conditionnelle.


##### 2.
Le vecteur $Z$ est composé des gaussiennes rapportant les points non observés connaissant les $n-p$ observations.  La loi de probabilité des $p$ premières valeurs observées sachant les $n-p$ observées $(V=v)$ suit la loi de densité suivante : 
$$
f_{Z | V=v}(y)=\frac{1}{(2 \pi)^{k / 2} \sqrt{\operatorname{det}\left(C S_{Z}\right)}}\left.\exp \left(-\frac{1}{2}(z-\psi(v))^{t} C S_{Z}^{-1}(z-\psi(v))\right)\right)
$$

Autrement dit la variable aléatoire $Z$ sachant $V=v$ suit une gaussienne d'espérance :

$$
m_{Z | V=v}=\psi(v)=m_{Z}+C_{Z, V} C_{V}^{-1}\left(v-m_{V}\right)
$$


et de matrice de covariance : 
$$
C S_{Z}=C_{Z}-C_{Z, V} C_{V}^{-1} C_{V, Z}
$$


##### 3.
Si Y est un vecteur de composantes gausiennes toutes indépendantes, toutes d'espérance nulle et de variance 1, alors d'après le cours (proba V)
le vecteur Z = m + RY est également un vecteur gaussien, d'espérence m et de matrice de Covariance R. 


##### 4. 
Nous allons simuler notre expérience en 3 étapes :
Dans un premier nous générons U, une variable aléatoire suivant une loi uniforme sur ]0, 1].
Puis grâce à la formule du cours, en posant $x_i=\sqrt{-2ln(U)}cos(2\pi U)$ on obtient X une variable aléatoire suivant une loi normale centrée réduite. (On répète cette opération n-p fois pour obtenir un vecteur de dimension n-p des gaussiennes). $X=\begin{pmatrix} ...\\ x_i \\ ... \end{pmatrix}$ 
Finalement en posant $Z = m + LX$ avec m l'espérence conditonnelle de Z sachant X, et L la décomposotion de Cholesky de la matrice de covariance conditionelle.

# Données du problème
Conventionnellement, $A$ est l'origine, $B=500$, $N=100$.

Les données $$\begin{array}{c|r}i & z(x_i)\\
\hline
0 & 0\\
20 & -4\\
40 & -12.8\\
60 & -1\\
80 & -6.5\\
100 & 0\end{array}$$

L'espérance de chaque composante du vecteur aléatoire $\mathbf{Z}$ est donnée par $\mu=-5.$

La fonction $C$ est définie par $$C(h)=\sigma^2 e^{-|h|/a},$$

où $|h|$ correspond à la distance entre deux points, $a=50$ et $\sigma^2=12$.


# Implémentation

## Préambule

In [ ]:
#Chargement de dépendances

import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv, cholesky

#Discrétisation
A=0
B=500
N=101 #Nombre de points de discrétisation
Delta = (B-A)/(N-1)
discretization_indexes = np.arange(N)
discretization = discretization_indexes*Delta
#Paramètres du modèle

mu=-5
a = 50
sigma2 = 12

#Données

obs_ind = [0,20,40,60,80,100]
depth = np.array([0,-4,-12.8,-1,-6.5,0])

#Indices des composantes correspondant aux observations et aux componsantes non observées

unk_ind=list(set(discretization_indexes)-set(obs_ind))


## Questions

1. Ecrire une fonction qui prend en argument la distance entre les points, le paramètre $a$, et le paramètre $\sigma^2$, et qui retourne la covariance entre deux points.
On pourra fournir une matrice de distance à cette fonction. Dans ce cas, la fonction renverra la matrice de covariance.

In [ ]:
def C(h, var, a):
    return var*np.exp(-h/a)

#On vectorise la fonction pour s'assurer le bon fonctionnement avec les matrices
C = np.vectorize(C)

2. Calculer la matrice de distance.

In [ ]:
def matrice_distance(N, Delta):
    res = np.zeros((N, N))
    for i in range(N):
        for j in range(i):
            rij = Delta * (i-j)
            res[i,j] = rij
            res[j,i] = rij
    return res

3. Calculer la matrice de covariance du vecteur $\mathbf{Z}=(Z(x_0),\dots,Z(x_N))$.

In [ ]:
CZ = C(matrice_distance(N, Delta), sigma2, a)
print(CZ)

4. Extraire les 3 matrices de covariance suivantes :

 * entre les observations

 * entre les observations et les inconnues

 * entre les inconnues


In [ ]:
Cobs = np.array([[ CZ[i][j] for j in obs_ind] for i in obs_ind])
Cboth = np.array([[ CZ[i][j] for i in obs_ind] for j in unk_ind])
Cunk = np.array([[ CZ[i][j] for j in unk_ind] for i in unk_ind])

print(Cobs)
print(Cboth)
print(Cunk)

5. Calculer l'espérance conditionnelle des composantes non observées connaissant les observations et la représenter avec les données.

In [ ]:
fig, ax = plt.subplots()

InvCobs = inv(Cobs)
m_v = [-5]*len(obs_ind)
m_z = [-5]*len(unk_ind)

CondEsp = m_z + np.dot((Cboth @ InvCobs), depth - m_v)

ax.scatter(unk_ind, CondEsp)
ax.scatter(obs_ind, depth)
ax.set_title("Esperance conditionelle en fonction de la position :")

6. Calculer la matrice de variance conditionnelle et tracer sa diagonale (variance conditionnelle) en fonction de la position. Commenter.

In [ ]:
fig, ax = plt.subplots()

MatVC = Cunk - (Cboth @ InvCobs) @ Cboth.T
diag = np.diag(MatVC)
 
VarCond = [(0 if i%20 == 0 else diag[i - (i//20 + 1)]) for i in range (101)]

ax.plot(VarCond)
ax.set_title("Variance conditionelle en fonction de la position :")

On remarque donc que plus on s'éloigne des points d'observations, moins l'estimation est précise. C'est un résultat attendu.

7. Effectuer une simulation conditionnelle. Sur un même graphique, tracer la simulation ainsi que les données et l'espérance conditionnelle. Commenter.

In [ ]:
def simulation():
    y = np.random.randn(95)
    R = cholesky(MatVC)
    Z = CondEsp + np.dot(R,y)
    for i in range(6):
        Z = np.insert(Z,obs_ind[i],depth[i])
    return Z

sim = simulation()
fig, ax = plt.subplots()
ax.plot(sim)
ax.scatter(obs_ind, depth)
ax.plot(unk_ind, CondEsp)
ax.set_title("Profondeur en fonction de la position :")
ax.legend(["Simulation", "Esperance Conditionelle", "Valeurs Connues"])

La simulation reste proche du de l'ésperance, nous avons donc bien réalisé un conditionnement.

8. Ecrire une fonction qui calcule la longueur du câble en fonction du vecteur des profondeurs et du pas de discrétisation.

In [ ]:
def length(depth, dx):
    res = 0
    for i in range(len(depth)-1):
        res += np.sqrt((depth[i+1]-depth[i])**2+dx**2)
    return res

9. Utiliser cette fonction pour calculer la longueur du câble à partir de 100 simulations. Comparer l'espérance conditionnelle (estimée) de la longueur avec la longueur de l'espérance conditionnelle.

In [ ]:
def lenBySim(n):
    Len = []
    for i in range(n) :
        Z = simulation()
        Len.append(length(Z, Delta))
    return Len

n = 100 
Lens = lenBySim(n)
lenSim = np.mean(Lens)
print("Apres {:} simulations, nous obtenons une estimation de la longueur du cable de {:3n}m.".format(n, lenSim))

10. Représenter la suite $M_n$ des moyennes des longueurs de câbles en fonction du nombre de simulations. Commenter.

In [ ]:
nbSim = list(range(1,101))
lenMean = []
for n in nbSim:
    lenMean.append(np.mean(lenBySim(n)))
    
print(lenMean)
fig, ax = plt.subplots()
ax.plot(nbSim, lenMean)
ax.set_title("Moyenne des longueurs de cable en fonction du nombre de simulations :")

Une simulation étant une variable aléatoire, il etait prévisible de voir les moyennes des longueurs converger vers une valeur apres de nombreuses simulations : ici 522.3m. C'est donc l'espérance donné par notre simulation.

11. Représenter l'histogramme des longueurs de câbles générées.

In [ ]:
Lens = lenBySim(100)

fig, ax = plt.subplots()
ax.hist(Lens)
ax.set_title("Hisogramme des longueurs données par simulations :")

12. Donner un intervalle de confiance à 95% de la longueur du câble par 2 méthodes différentes. Commenter.

In [ ]:
#Première methode : Prendre la moyenne puis l'ecart type, les bornes de l'intervalle sont cette première plus ou moins 2 fois l'ecart type
#Deuxieme methode : On prend 95% des valeurs, si le nombre de simulations est assez grand

Lens = sorted(Lens)

print("Premier intervalle de confiance : [{},{}]".format(np.mean(Lens)-2*np.std(Lens), np.mean(Lens)+2*np.std(Lens)))
print("Premier intervalle de confiance : [{},{}]".format(Lens[2],Lens[-3]))

13. Donner une estimation de la probabilité que la longueur du câble dépasse 525 m.

In [ ]:
Lens = lenBySim(100)
Lens = np.array(sorted(Lens))
print("Un estimation de la probabilite que la longueur du cable dépasse 525m est : {}.".format(np.argwhere(Lens >= 525).size/1000))

14. Reprendre les questions précédentes avec 1000, 10000 puis 100000 simulations. Commenter.

In [ ]:
for n in [1000, 10000, 100000]:
    print(f"\tPour {n} simulations :")
    Lens = lenBySim(n)
    Lens = np.array(sorted(Lens))
    lenSim = np.mean(Lens)
    print("Apres {:} simulations, nous obtenons une estimation de la longueur du cable de {:3n}m.".format(n, lenSim))
    fig, ax = plt.subplots()
    ax.hist(Lens)
    ax.set_title(f"Hisogramme des longueurs données par simulations :")
    print("Premier intervalle de confiance : [{},{}]".format(np.mean(Lens)-2*np.std(Lens), np.mean(Lens)+2*np.std(Lens)))
    print("Premier intervalle de confiance : [{},{}]".format(Lens[2],Lens[-3]))
    print("Un estimation de la probabilite que la longueur du cable dépasse 525m est : {}.\n".format(np.argwhere(Lens >= 525).size/n))

### Conclusion
On remarque qu'augmenter grandement le nombre de simulations n'est pas nécessaire. La précision est suffisament grande avec peu de simulations et elle ne s'améliore pas tant que ca avec le nombre.